# Tidy3D mode solver

In [ ]:
import gdsfactory.simulation.gtidy3d as gt

In [ ]:
strip = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, t_wg=0.22, t_slab=0.0)
strip

In [ ]:
strip.plot_Ex(0) # TE
strip.plot_Ey(1) # TM

In [ ]:
rib = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, t_wg=0.22, t_slab=0.15)
rib.plot_index()

In [ ]:
rib.plot_Ex(index=0)
rib.plot_Ey(index=0)

## Bend modes

You can compute bend modes specifying the bend radius.

In [ ]:
strip_bend = gt.modes.Waveguide(wavelength=1.55, wg_width=0.5, t_wg=0.22, t_slab=0.0, bend_radius=3)

In [ ]:
# lets plot the fundamental TE mode
strip_bend.plot_Ex(0)
strip_bend.plot_Ey(0)

In [ ]:
# lets plot the fundamental TM mode
strip_bend.plot_Ex(1)
strip_bend.plot_Ey(1)

In [ ]:
strip.pickle_dump('strip.pkl') # save Waveguide with mode data to file

In [ ]:
s = gt.modes.pickle_load('strip.pkl') # Load Waveguide from disk with modes
s.plot_Ex()

## Bend loss

You can also compute the losses coming from the mode missmatch from the bend into a straight waveguide.
To compute the bend loss due to mode missmatch you can calculate the mode overlap of the original mode and the bent mode.
Because there are two mode missmatch interfaces the total loss due to mode missmatch will be double.

In [ ]:
import gdsfactory.simulation.gtidy3d as gt
import matplotlib.pyplot as plt

r, integral = gt.modes.sweep_bend_loss()
plt.plot(r, 2*integral/max(integral), ".")
plt.xlabel('bend radius (um)')
plt.ylabel('Transmission')
plt.show()